In [4]:
#id-2 - [1,0], id_7 - [0,1]
import tensorflow as tf 

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [110]:
data_x, data_y = [], []
num_class = 9
img_h=250
img_w=250
img_c=3
img_length = 250*250*3

id_list = [0,1,2,3,4,5,6,7,8]

for i in range(len(id_list)):
    part_x, part_y = [], []
    folder = '../pixiv/quar_'+str(id_list[i])
    for img_file in os.listdir(folder):
#         print(folder+'/'+img_file)
        img = np.array(Image.open(folder+'/'+img_file))
#         img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
        part_x.append(img/255.0)
        
        label = [0 for j in range(0, num_class)]
        label[i]=1
        part_y.append(label)   
        if(len(part_x)>=590):
            break
    print(len(part_x))
    data_x.extend(part_x)
    data_y.extend(part_y)
    print(folder+' is completed.')

print(len(data_x))
print(len(data_y))

590
../pixiv/quar_0 is completed.
590
../pixiv/quar_1 is completed.
590
../pixiv/quar_2 is completed.
590
../pixiv/quar_3 is completed.
590
../pixiv/quar_4 is completed.
590
../pixiv/quar_5 is completed.
590
../pixiv/quar_6 is completed.
590
../pixiv/quar_7 is completed.
590
../pixiv/quar_8 is completed.
5310
5310


In [111]:
data_x = np.array(data_x, dtype='float32')
data_y = np.array(data_y, dtype='float32')

print(data_x[:3])
print(data_y[:3])

[[[[ 0.53725493  0.46666667  0.51372552]
   [ 0.54901963  0.40000001  0.4509804 ]
   [ 0.58039218  0.32156864  0.36862746]
   ..., 
   [ 0.57254905  0.7019608   0.83137256]
   [ 0.54901963  0.68235296  0.82745099]
   [ 0.56078434  0.69411767  0.84313726]]

  [[ 0.51764709  0.38039216  0.45882353]
   [ 0.56862748  0.35686275  0.44705883]
   [ 0.63921571  0.32941177  0.41176471]
   ..., 
   [ 0.49803922  0.63529414  0.75294119]
   [ 0.57254905  0.70980394  0.83529413]
   [ 0.58431375  0.72549021  0.86666667]]

  [[ 0.53333336  0.33333334  0.45490196]
   [ 0.57254905  0.30980393  0.43921569]
   [ 0.65490198  0.29803923  0.41960785]
   ..., 
   [ 0.01568628  0.16078432  0.26274511]
   [ 0.37254903  0.51764709  0.62352943]
   [ 0.50588238  0.65098041  0.7647059 ]]

  ..., 
  [[ 0.49803922  0.68235296  0.85490197]
   [ 0.49803922  0.68235296  0.85490197]
   [ 0.49803922  0.68235296  0.85490197]
   ..., 
   [ 0.35686275  0.52156866  0.61960787]
   [ 0.52549022  0.65098041  0.74117649]
   [ 0.

In [112]:
order = [i for i in range(len(data_x))]
random.shuffle(order)

print(len(order))
n=int(len(data_x)*9/10)
print(n)

train_x = [data_x[i] for i in order[0:n]]
train_y = [data_y[i] for i in order[0:n]]

test_x = [data_x[i] for i in order[n:]]
test_y = [data_y[i] for i in order[n:]]

print(len(train_x))
print(len(test_y))

5310
4779
4779
531


In [131]:
learning_rate=0.003

graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    is_training=True

    def tf_variable(shape, name=None):
        return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name=name)

    def dense_connect(x, shape):
        w = tf_variable(shape)
        b = tf.Variable(tf.zeros([shape[1]]))
        return tf.matmul(x, w) + b

    def batch_norm(inputs, is_training,is_conv_out=True,decay = 0.999):
        scale = tf.Variable(tf.ones([inputs.get_shape()[-1]]))
        beta = tf.Variable(tf.zeros([inputs.get_shape()[-1]]))
        pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]), trainable=False)
        pop_var = tf.Variable(tf.ones([inputs.get_shape()[-1]]), trainable=False)
        if is_training:
            if is_conv_out:
                batch_mean, batch_var = tf.nn.moments(inputs,[0,1,2])
            else:
                batch_mean, batch_var = tf.nn.moments(inputs,[0])   

            train_mean = tf.assign(pop_mean,
                                   pop_mean * decay + batch_mean * (1 - decay))
            train_var = tf.assign(pop_var,
                                  pop_var * decay + batch_var * (1 - decay))
            with tf.control_dependencies([train_mean, train_var]):
                return tf.nn.batch_normalization(inputs,
                    batch_mean, batch_var, beta, scale, 0.001)
        else:
            return tf.nn.batch_normalization(inputs,
                pop_mean, pop_var, beta, scale, 0.001)

    def conv2d_with_batch_norm(x, filter_shape, stride):
        filter_ = tf_variable(filter_shape)
        conv = tf.nn.conv2d(x, filter=filter_, strides=[1, stride, stride, 1], padding="SAME")
        normed=batch_norm(conv, is_training)
        return  tf.nn.relu(normed)


    def conv2d(x, filter_shape, stride):
        out_channels = filter_shape[3]
        conv = tf.nn.conv2d(x, filter=tf_variable(filter_shape), strides=[1, stride, stride, 1], padding="SAME")
        bias = tf.Variable(tf.zeros([out_channels]), name="bias")
        return tf.nn.relu(tf.nn.bias_add(conv,bias))


    def max_pool(x):
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    def residual_block(x, out_channels, down_sample, projection=False):
        in_channels = x.get_shape().as_list()[3]
        if down_sample:
            x = max_pool(x)

        output = conv2d_with_batch_norm(x, [3, 3, in_channels, out_channels], 1)
        output = conv2d_with_batch_norm(output, [3, 3, out_channels, out_channels], 1)

        if in_channels != out_channels:
            if projection:
                # projection shortcut
                input_ = conv2d(x, [1, 1, in_channels, out_channels], 2)
            else:
                # zero-padding
                input_ = tf.pad(x, [[0,0], [0,0], [0,0], [0, out_channels - in_channels]])
        else:
            input_ = x

        return output + input_


    def residual_group(name,x,num_block,out_channels):
        assert num_block>=1,'num_block must greater than 1'
        with tf.variable_scope('%s_head'%name):
            output = residual_block(x, out_channels, True)
        for i in range (num_block-1):
            with tf.variable_scope('%s_%d' % (name,i+1)):
                output = residual_block(output,out_channels, False)
        return output


    def residual_net(inpt):
        with tf.variable_scope('conv1'):
            output = conv2d(inpt, [3, 3, 3, 16], 1)
            print(output.shape)
        output=residual_group('conv2', x=output,num_block=2,out_channels=16)
        print(output.shape)
        output=residual_group('conv3', x=output,num_block=2,out_channels=32)
        print(output.shape)
        output=residual_group('conv4', x=output,num_block=2,out_channels=64)
        print(output.shape)
        with tf.variable_scope('fc'):
            output=max_pool(output)
            print(output.shape)
            shape=output.get_shape().as_list()
            i_shape=shape[1]*shape[2]*shape[3]
            print(i_shape)
            output=tf.reshape(output,[-1,i_shape])
            print(output.shape)
            return dense_connect(output, [i_shape, num_class])

    x = tf.placeholder("float", [None, img_h, img_w, img_c])#[batch_size,width,height,channels]
    y = tf.placeholder("float", [None, num_class])#[batch_size,num_classes]

#     x_image = tf.reshape(x, [-1,250,250,3])
    pred = residual_net(x)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#     accuracyte = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

(?, 250, 250, 16)
(?, 125, 125, 16)
(?, 63, 63, 32)
(?, 32, 32, 64)
(?, 16, 16, 64)
16384
(?, 16384)


In [134]:
batch_size = 256
training_iters=401
batch_start=0

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    step = 1
    print('Initialized')
    for step in range(training_iters): 
        if(batch_start+batch_size>=len(train_x)): 
            order = [i for i in range(len(train_x))]
            random.shuffle(order)
            train_x = [train_x[i] for i in order]
            train_y = [train_y[i] for i in order]
            batch_start = 0   
        feed_x = train_x[batch_start:batch_start+batch_size]
        feed_y = train_y[batch_start:batch_start+batch_size]
        
#         sess.run(optimizer, feed_dict={inputs: feed_x, labels: feed_y})
        _, l, accur = sess.run([optimizer, cost, accuracy], feed_dict = {x: feed_x, y: feed_y})
        if(step % 5 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('accuarcy is ', accur)
            test_loss, test_accur = sess.run([cost, accuracy], feed_dict={x: test_x, y: test_y})
            print('test loss, ', test_loss, ' test accur is ', test_accur)
            
print("train finished")

Initialized
Loss at step 0: 65.163017
accuarcy is  0.117188
test loss,  69.3474  test accur is  0.154426
Loss at step 5: 68.395706
accuarcy is  0.226562
test loss,  90.8008  test accur is  0.124294
Loss at step 10: 33.811031
accuarcy is  0.226562
test loss,  52.6246  test accur is  0.177024
Loss at step 15: 14.541766
accuarcy is  0.289062
test loss,  26.8141  test accur is  0.186441
Loss at step 20: 7.513888
accuarcy is  0.277344
test loss,  17.1973  test accur is  0.158192
Loss at step 25: 4.811194
accuarcy is  0.273438
test loss,  15.7289  test accur is  0.150659
Loss at step 30: 2.749368
accuarcy is  0.320312
test loss,  15.2017  test accur is  0.141243
Loss at step 35: 2.694733
accuarcy is  0.308594
test loss,  15.6243  test accur is  0.116761
Loss at step 40: 2.018274
accuarcy is  0.304688
test loss,  14.5431  test accur is  0.141243
Loss at step 45: 1.987470
accuarcy is  0.316406
test loss,  14.8419  test accur is  0.146893
Loss at step 50: 1.740154
accuarcy is  0.40625
test loss